# A benchmark for time-frequency denoising/ detecting.

## What is this benchmark?
A benchmark is a comparison between different methods when running an standarized test. The goal of this benchmark is to compare different methods for denoising / detecting a signal based on the properties of the time-frequency plane. This particular benchmark has been created for evaluating the performance of techniques based on the zeros of the spectrogram and to contrast them with more traditional methods, like those based on the ridges in the time-frequency plane.

With the purpose of making this benchmark more useful, the methods to compare, the tests, and the performance evaluation functions where concived as different modules, so that one can test newer methods in the future without modifing the tests. The only restrictions this pose is that the methods to test should satisfy some requirements regarding the shape of their input an output parameters. The tests, comprising the signals used and the performance evaluation functions, are encapsulated in the class `Benchmark`.

In order to compare different methods with possibly different parameters, we need to set up a few things before running the benchmark. A `Benchmark` object receives some input parameters to configure the test:
- `task`: This could be `'denoising'` or `'detecting'`. The first one compute the quality reconstruction factor (QRF) using the output of the method, whereas the second simply consist in detecting wheter a signal is present or not.
- `N`: The length of the simulation, i.e. how many samples should the signals have.
- `methods`: A dictionary of methods. Each entry of this dictionary corresponds to the function that implements each of the desired methods.
- `parameters`: A dictionary of parameters. Each entry of this dictionary corresponds to an array of tuples to pass as input parameters to each method. In order to know which parameters should be passed to each methods, the keys of this dictionary should be the same as those corresponding to the individual methods in the corresponding dictionary.
- `SNRin`: A list or tuple of values of SNR to test.
- `repetitions`: The number of times the experiment should be repeated with different realizations of noise.

## A dummy test 
First let's define a dummy method for testing. Methods should receive an `M`x`N` array of noisy signals, where `M` is the number of signals, and `N` is the number of time samples. Additionally, it should receive a second parameter `params` to test different combinations of parameters of the method. The shape of the output depends on the task (denoising or detecting). So the firm of a method should be the following:

 `output = a_method(noisy_signals,params) `.

If one set `task='denoising'`, the shape of `output` should be also `M`x`N`, i.e. the same shape as the input parameter `noisy_signals`, whereas if `task='detecting'`, the output should be boolean (`0` or `False` for no signal, and `1` or `True` otherwise).

After this, we should create a dictionary of methods to pass the `Benchmark` object at the moment of instantiation.

In [1]:
import numpy as np
from numpy import pi as pi
import pandas as pd
from benchmark_demo.Benchmark import Benchmark

def a_method(noisy_signals, params):
    # If additional input parameters are needed, they can be passed in a tuple using params and then parsed.
    results = noisy_signals 
    return results

# Create a dictionary of the methods to test.
my_methods = {
    'Method 1': a_method, 
    'Method 2': a_method
    }

Let us now create a dictionary of the parameters to pass to each method. The parameters should be given in a tuple of tuples, so that each internal tuple is passed as a single additional parameter to the method (this latter should implement how to parse the parameters from this tuple). *Remember that the keys of this dictionary should be the same as those of the methods dictionary*.

In [2]:
my_parameters = {
    'Method 1': [[3, 4, True, 'all'],[2, 1, False, 'one']], # Each tuple is passed as a parameter that the method should parse internally.
    'Method 2': [[3, 4, True, 'all'],[2, 1, False, 'one']]
    }

Now we are ready to instantiate a `Benchmark` object, run a test using the proposed methods and parameters. The benchmark constructor receives a name of a task (which defines the performance function for the test), a dictionary of the methods to test, the desired length of the signals used in the simulation, a dictionary of different parameters that should be passed to the methods, an array with different values of SNR to test, and the number of repetitions that should be used for each test.

In [3]:
my_benchmark = Benchmark(task = 'denoising', methods = my_methods, N = 256, parameters = my_parameters, SNRin = [40,50], repetitions = 3)
my_results = my_benchmark.runTest() # Run the test. my_results is a dictionary with the results for each of the variables of the simulation.

Running benchmark...
The test has finished.


Now we have the results of the test in a nested dictionary called `my_results`. In order to get the results in a human-readable way using a DataFrame, and also for further analysis and reproducibility, we could use the method `getResults()`.

In [4]:
df = my_benchmark.getResults() # This formats the results on a DataFrame
df.head()

,Method,Parameter,Signal_id,Repetition,40,50
6,Method 1,Params0,cosChirp,0,40.000000,50.000000
7,Method 1,Params0,cosChirp,1,40.000000,50.000000
8,Method 1,Params0,cosChirp,2,40.000000,50.000000
12,Method 1,Params0,crossedLinearChirps,0,39.999997,49.999967
13,Method 1,Params0,crossedLinearChirps,1,39.999997,49.999967


As we can see, the table show the results ordered by columns. The first column corresponds to the method identification, and the values are taken from the keys of the dictionary of methods. The second column enumerates the parameters used (more on this on the next section). The third column corresponds to the signal identification, using the signal identification values from the `SignalBank` class. The next column shows the number of repetition of the experiment. Finally, the remaining columns show the results obtained for the SNR values used for each experiment. Since `task = 'denoising'`, these values correspond to the QRF computed as `QRF = 10*np.log10(E(s)/E(s-sr))`, where `E(x)` is the energy of `x`, and `s` and `sr` are the noiseless signal and the reconstructed signal respectively.

## Passing different parameters to the methods.
It is common that a method depends on certain input parameters (thresholds, multiplicative factors, etc). Therefore, it would be useful that the tests could also be repeated with different parameters, instead of creating different versions of one method. We can pass an array of parameters to our method provided it parses them internally. For instance, let's consider a function that depends on two thresholds `thr1` and `thr2`:

In [5]:
def a_function(signal, thr1, thr2):
    signal_out = signal
    signal_out[np.where((thr2>signal) & (signal>thr1))] = 1
    return signal_out

Now lets create the method and the dictionary for our benchmark. Notice that the method function should parse the parameters in `params`. Then we can define the diferent combinations of parameters using the corresponding dictionary:

In [6]:
def another_method(noisy_signals,params):
    output_signals = np.zeros(noisy_signals.shape)
    # The method should parse the parameters accordingly
    thr1 = params[0]
    thr2 = params[1]
    output = [a_function(signal, thr1, thr2) for signal in noisy_signals]
    output = np.array(output)
    return output
        

# Create a dictionary of the methods to test.
my_methods = {
    'Method 1': another_method, 
    }

# Create a dictionary of the different combinations of the thresholds to test:
thr1 = np.arange(0.1,0.5,0.2) # Some values for the thresholds.
thr2 = np.arange(4,6,1)
my_parameters = {
    'Method 1': [[t1,t2] for t1 in thr1 for t2 in thr2], # Remember the keys of this dictionary should be same as the methods dictionary.
}

print((my_parameters['Method 1']))

[[0.1, 4], [0.1, 5], [0.30000000000000004, 4], [0.30000000000000004, 5]]


So now we have four combinations of the input parameters for the method `another_method()`, which will be passed one by one to the method, so that all the experiments will be carried out for each of these combinations. Lets set the benchmark again and run a test using this new configuration of methods and parameters. After that, use the the `Benchmark` class method `getResults()` to obtain a table with the results.

In [7]:
my_benchmark = Benchmark(task = 'denoising', methods = my_methods, N = 256, parameters = my_parameters, SNRin = [40,50], repetitions = 3)
my_results = my_benchmark.runTest() # Run the test. my_results is a dictionary with the results for each of the variables of the simulation.
df = my_benchmark.getResults() # This formats the results on a DataFrame
df.shape
df

Running benchmark...
The test has finished.


,Method,Parameter,Signal_id,Repetition,40,50
3,Method 1,Params0,cosChirp,0,7.650968,7.840755
4,Method 1,Params0,cosChirp,1,7.839270,7.840752
5,Method 1,Params0,cosChirp,2,7.834547,7.840756
6,Method 1,Params0,crossedLinearChirps,0,6.765070,6.667951
7,Method 1,Params0,crossedLinearChirps,1,6.666834,6.766290
...,...,...,...,...,...,...
70,Method 1,Params3,multiComponentHarmonic,1,4.376974,4.377947
71,Method 1,Params3,multiComponentHarmonic,2,4.377003,4.377944
66,Method 1,Params3,sharpAttackCos,0,9.296257,9.272040
67,Method 1,Params3,sharpAttackCos,1,9.295279,9.271967


As we can see, the experiments has been repited for every combinations of parameters, listed in the second column of the table as `Param#` where `#` is the number corresponding to one group of input parameters.